In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [3]:
data = spark.read.csv('/FileStore/tables/Ecommerce_Customers.csv', header=True, inferSchema=True)

In [4]:
data.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)

In [5]:
data.head(1)

Out[ 6 ]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)]

In [6]:
for item in data.head(2)[0]:
  print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005

In [7]:
for item in data.head(2)[1]:
  print(item)

hduke@hotmail.com
4547 Archer CommonDiazchester, CA 06566-8576
DarkGreen
31.92627202636016
11.109460728682564
37.268958868297744
2.66403418213262
392.2049334443264

In [8]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [9]:
data.columns

Out[ 10 ]: 
['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [10]:
assembler = VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'], outputCol='features')

In [11]:
output = assembler.transform(data)

In [12]:
output.printSchema()

root
-- Email: string (nullable = true)
-- Address: string (nullable = true)
-- Avatar: string (nullable = true)
-- Avg Session Length: double (nullable = true)
-- Time on App: double (nullable = true)
-- Time on Website: double (nullable = true)
-- Length of Membership: double (nullable = true)
-- Yearly Amount Spent: double (nullable = true)
-- features: vector (nullable = true)

In [13]:
output.head(1)

Out[ 19 ]: [Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [14]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [15]:
final_data.show(2)

+--------------------+-------------------+
 features|Yearly Amount Spent|
+--------------------+-------------------+
[34.4972677251122...| 587.9510539684005|
[31.9262720263601...| 392.2049334443264|
+--------------------+-------------------+
only showing top 2 rows

In [16]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [17]:
train_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 332|
 mean| 499.19166580903175|
 stddev| 82.5640694183741|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [18]:
test_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 168|
 mean| 499.55586952795716|
 stddev| 72.70407556470464|
 min| 298.76200786180766|
 max| 684.1634310159512|
+-------+-------------------+

In [19]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol = 'Yearly Amount Spent')

In [20]:
lr_model = lr.fit(train_data)

In [21]:
test_result = lr_model.evaluate(test_data)

In [22]:
test_result.residuals.show()

+-------------------+
 residuals|
+-------------------+
-11.963637301138874|
 -0.503156087553009|
 -4.251564326597361|
-4.6111013894569055|
-13.082146294773509|
 -8.393360233031672|
 17.290479218119856|
 -4.741028008078047|
 -8.361340599344771|
-7.7631675584382265|
 17.928886260343916|
 17.282128432444154|
 -5.158238385769266|
 -6.018807757723778|
-1.9575490699814964|
 -9.888783594945323|
 -5.515578877887208|
-13.966651404597087|
 8.134832375420615|
 -18.04330805337372|
+-------------------+
only showing top 20 rows

In [23]:
test_result.rootMeanSquaredError

Out[ 38 ]: 10.02995559118273

In [24]:
test_result.r2

Out[ 39 ]: 0.9808542088402898

In [25]:
final_data.describe().show()

+-------+-------------------+
summary|Yearly Amount Spent|
+-------+-------------------+
 count| 500|
 mean| 499.3140382585909|
 stddev| 79.3147815497068|
 min| 256.67058229005585|
 max| 765.5184619388373|
+-------+-------------------+

In [26]:
unlabeled_data = test_data.select(['features'])

In [27]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
[30.3931845423455...|
[30.5743636841713...|
[30.8364326747734...|
[30.8794843441274...|
[31.0662181616375...|
[31.1280900496166...|
[31.3123495994443...|
[31.4252268808548...|
[31.4474464941278...|
[31.5147378578019...|
[31.6005122003032...|
[31.6098395733896...|
[31.6253601348306...|
[31.7242025238451...|
[31.8124825597242...|
[31.8279790554652...|
[31.8745516945853...|
[31.9365486184489...|
[31.9549038566348...|
[31.9563005605233...|
+--------------------+
only showing top 20 rows

In [28]:
predictions = lr_model.transform(unlabeled_data)

In [29]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[30.3931845423455...| 331.8925071043325|
[30.5743636841713...|442.56756984561866|
[30.8364326747734...|471.75346475358697|
[30.8794843441274...| 494.8177013743116|
[31.0662181616375...|462.01543950244786|
[31.1280900496166...| 565.6460469800863|
[31.3123495994443...|446.30093880982076|
[31.4252268808548...| 535.50774666284|
[31.4474464941278...| 426.9640826945688|
[31.5147378578019...|497.57565555489964|
[31.6005122003032...| 461.243965230753|
[31.6098395733896...| 427.263421218664|
[31.6253601348306...|381.49513914269346|
[31.7242025238451...| 509.4066950456843|
[31.8124825597242...| 394.7678940537787|
[31.8279790554652...|449.89153114188684|
[31.8745516945853...| 397.8008231241547|
[31.9365486184489...| 441.1660362999253|
[31.9549038566348...|431.86304756450636|
[31.9563005605233...| 565.1692398005725|
+--------------------+------------------+
only showing top 20 rows

In [30]:
#Real project
data = spark.read.csv('/FileStore/tables/cruise_ship_info.csv', header=True, inferSchema=True)

In [31]:
data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)

In [32]:
data.head(2)

Out[ 5 ]: 
[Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55),
 Row(Ship_name='Quest', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)]

In [33]:
data.show(2)

+---------+-----------+---+------------------+----------+------+------+-----------------+----+
Ship_name|Cruise_line|Age| Tonnage|passengers|length|cabins|passenger_density|crew|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
 Journey| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
 Quest| Azamara| 6|30.276999999999997| 6.94| 5.94| 3.55| 42.64|3.55|
+---------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 2 rows

In [34]:
data.head(2)[0]

Out[ 6 ]: Row(Ship_name='Journey', Cruise_line='Azamara', Age=6, Tonnage=30.276999999999997, passengers=6.94, length=5.94, cabins=3.55, passenger_density=42.64, crew=3.55)

In [35]:
data.columns

Out[ 7 ]: 
['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [36]:
from pyspark.ml.feature import StringIndexer

In [37]:
indexer = StringIndexer(inputCol = 'Cruise_line', outputCol = 'Cruise_cat')

In [38]:
indexed_data = indexer.fit(data).transform(data)

In [39]:
indexed_data.printSchema()

root
-- Ship_name: string (nullable = true)
-- Cruise_line: string (nullable = true)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- Cruise_cat: double (nullable = false)

In [40]:
final_data = indexed_data.select(['Cruise_cat', 'Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density', 'crew'])

In [41]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [42]:
assembler = VectorAssembler(inputCols=['Cruise_cat', 'Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density'], outputCol='features')

In [43]:
output = assembler.transform(final_data)

In [44]:
output.printSchema()

root
-- Cruise_cat: double (nullable = false)
-- Age: integer (nullable = true)
-- Tonnage: double (nullable = true)
-- passengers: double (nullable = true)
-- length: double (nullable = true)
-- cabins: double (nullable = true)
-- passenger_density: double (nullable = true)
-- crew: double (nullable = true)
-- features: vector (nullable = true)

In [45]:
dataset = output.select(['features', 'crew'])

In [46]:
dataset.show(2)

+--------------------+----+
 features|crew|
+--------------------+----+
[16.0,6.0,30.2769...|3.55|
[16.0,6.0,30.2769...|3.55|
+--------------------+----+
only showing top 2 rows

In [47]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(labelCol='crew')

In [48]:
train_data, test_data = dataset.randomSplit([0.7, 0.3])

In [49]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 113|
 mean|7.731858407079651|
 stddev|3.727040081014253|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [50]:
test_data.describe().show()

+-------+------------------+
summary| crew|
+-------+------------------+
 count| 45|
 mean| 7.950666666666668|
 stddev|2.8989421268016806|
 min| 1.6|
 max| 13.6|
+-------+------------------+

In [51]:
lr_model = lr.fit(train_data)

In [52]:
test_result = lr_model.evaluate(test_data)

In [53]:
test_result.rootMeanSquaredError

Out[ 29 ]: 1.0344253017145895

In [54]:
train_data.describe().show()

+-------+-----------------+
summary| crew|
+-------+-----------------+
 count| 113|
 mean|7.731858407079651|
 stddev|3.727040081014253|
 min| 0.59|
 max| 21.0|
+-------+-----------------+

In [55]:
test_result.r2

Out[ 111 ]: 0.8660625601279829

In [56]:
test_result.rootMeanSquaredError

Out[ 33 ]: 1.0344253017145895

In [57]:
unlabeled_data = test_data.select(['features'])

In [58]:
predictions = lr_model.transform(unlabeled_data)

In [59]:
predictions.show()

+--------------------+------------------+
 features| prediction|
+--------------------+------------------+
[0.0,13.0,24.0,10...|1.7148667247749434|
[1.0,1.0,11.0,86....| 9.522595180720328|
[2.0,14.0,27.0,5....| 1.626937631538206|
[8.0,4.0,11.0,90....|10.000237879294204|
[9.0,4.0,12.0,77....| 8.706722712188244|
[11.0,0.0,16.0,78...| 8.219762132320165|
[11.0,7.0,36.0,16...| 3.280117184665328|
[12.0,0.0,11.0,13...|12.866421126748474|
[13.0,0.0,7.0,158...|13.634066438513711|
[13.0,1.0,6.0,110...|10.798994087590035|
[14.0,0.0,10.0,13...|12.871547143998804|
[24.0,0.0,16.0,74...| 8.690095577003445|
[26.0,3.0,5.0,86....| 9.236677978081039|
[30.0,17.0,14.0,8...| 9.247209050557503|
[33.0,1.0,18.0,70...| 8.597341737535418|
[34.0,3.0,27.0,53...|6.8524511239635935|
[38.0,7.0,5.0,133...| 12.87256215746453|
[42.0,6.0,17.0,77...| 8.528476555430231|
[45.0,2.0,10.0,91...| 9.317355681920542|
[53.0,0.0,22.0,73...| 8.911875450512024|
+--------------------+------------------+
only showing top 20 rows

In [60]:
from pyspark.sql.functions import corr

In [61]:
data.select(corr('crew', 'passengers')).show()

+----------------------+
corr(crew, passengers)|
+----------------------+
 0.9152341306065384|
+----------------------+

In [62]:
data.select(corr('crew', 'cabins')).show()

+------------------+
corr(crew, cabins)|
+------------------+
0.9508226063578497|
+------------------+

In [63]:
test_result.meanAbsoluteError

Out[ 38 ]: 0.7212217949575331